### Clustering Wrangle Exercises

#### Acquire

Zillow

For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

You will have a zillow.ipynb file and a helper file for each section in the pipeline.

In [195]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
pd.options.display.float_format = '{:20,.2f}'.format

import env

In [62]:
def get_db_url(database):
    url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/{database}'
    return url

def get_zillow():
    sql = '''
SELECT properties_2017.*, predictions_2017.logerror, 
predictions_2017.transactiondate, 
airconditioningtype.airconditioningdesc, architecturalstyletype.architecturalstyledesc,
buildingclasstype.buildingclassdesc, heatingorsystemtype.heatingorsystemdesc,
propertylandusetype.propertylandusedesc, storytype.storydesc, 
typeconstructiontype.typeconstructiondesc
FROM properties_2017
LEFT JOIN airconditioningtype USING(airconditioningtypeid)
LEFT JOIN architecturalstyletype USING(architecturalstyletypeid)
LEFT JOIN buildingclasstype USING(buildingclasstypeid)
LEFT JOIN heatingorsystemtype USING(heatingorsystemtypeid)
LEFT JOIN propertylandusetype USING(propertylandusetypeid)
LEFT JOIN storytype USING(storytypeid)
LEFT JOIN typeconstructiontype USING(typeconstructiontypeid)
INNER JOIN (SELECT parcelid, logerror, MAX(transactiondate) transactiondate 
FROM predictions_2017 GROUP BY parcelid, logerror) predictions_2017 USING(parcelid)
WHERE properties_2017.longitude IS NOT NULL 
AND properties_2017.latitude IS NOT NULL 
AND predictions_2017.transactiondate LIKE '2017%%';
    '''
    url = get_db_url('zillow')
    zillow_df = pd.read_sql(sql, url, index_col = 'id')
    return zillow_df

1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

 - Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
 - Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
 - Only include properties that include a latitude and longitude value.

In [91]:
#df = get_zillow()

In [65]:
#df.to_csv("zillow.csv")

In [129]:
df = pd.read_csv("zillow.csv")

2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77574 entries, 0 to 77573
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77574 non-null  int64  
 1   parcelid                      77574 non-null  int64  
 2   airconditioningtypeid         25006 non-null  float64
 3   architecturalstyletypeid      206 non-null    float64
 4   basementsqft                  50 non-null     float64
 5   bathroomcnt                   77574 non-null  float64
 6   bedroomcnt                    77574 non-null  float64
 7   buildingclasstypeid           15 non-null     float64
 8   buildingqualitytypeid         49808 non-null  float64
 9   calculatedbathnbr             76959 non-null  float64
 10  decktypeid                    614 non-null    float64
 11  finishedfloor1squarefeet      6035 non-null   float64
 12  calculatedfinishedsquarefeet  77374 non-null  float64
 13  f

In [67]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
parcelid,"77,574.00","13,006,297.98","3,478,043.16","10,711,855.00","11,538,192.50","12,530,505.00","14,211,242.75","167,689,317.00"
airconditioningtypeid,"25,006.00",1.81,2.97,1.00,1.00,1.00,1.00,13.00
architecturalstyletypeid,206.00,7.39,2.73,2.00,7.00,7.00,7.00,21.00
basementsqft,50.00,679.72,689.70,38.00,273.00,515.00,796.50,"3,560.00"
bathroomcnt,"77,574.00",2.30,1.00,0.00,2.00,2.00,3.00,18.00
bedroomcnt,"77,574.00",3.05,1.14,0.00,2.00,3.00,4.00,16.00
buildingclasstypeid,15.00,3.93,0.26,3.00,4.00,4.00,4.00,4.00
buildingqualitytypeid,"49,808.00",6.53,1.72,1.00,6.00,6.00,8.00,12.00
calculatedbathnbr,"76,959.00",2.32,0.98,1.00,2.00,2.00,3.00,18.00
decktypeid,614.00,66.00,0.00,66.00,66.00,66.00,66.00,66.00


3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [68]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = num_missing / rows
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': prcnt_miss})
    return cols_missing

In [69]:
nulls_by_col(df)

,num_rows_missing,percent_rows_missing
parcelid,0,0.00
airconditioningtypeid,52568,0.68
architecturalstyletypeid,77368,1.00
basementsqft,77524,1.00
bathroomcnt,0,0.00
...,...,...
buildingclassdesc,77559,1.00
heatingorsystemdesc,28005,0.36
propertylandusedesc,0,0.00
storydesc,77524,1.00


4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [70]:
def nulls_by_row(df):
    df2 = pd.DataFrame(df.isnull().sum(axis = 1), columns = ['num_cols_missing']).reset_index()\
    .groupby('num_cols_missing').count().reset_index().\
    rename(columns = {'id': 'num_rows'})
    df2['pct_cols_missing'] = df2.num_cols_missing/df.shape[1]
    return df2

In [71]:
nulls_by_row(df).sort_values(by = ['pct_cols_missing'], ascending = False)

,num_cols_missing,num_rows,pct_cols_missing
25,48,3,0.72
24,47,3,0.70
23,46,5,0.69
22,45,50,0.67
21,44,79,0.66
20,43,29,0.64
19,42,26,0.63
18,41,29,0.61
17,40,230,0.60
16,39,285,0.58


#### Prepare

1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [92]:
df.shape

(77574, 67)

In [169]:
# look at value counts for 'unitcnt'

df.unitcnt.value_counts(dropna = False)

                1.00    47412
NaN                     26872
                2.00     2031
                4.00      726
                3.00      529
               45.00        1
               42.00        1
                6.00        1
              237.00        1
Name: unitcnt, dtype: int64

In [170]:
# Restrict df to only properties that meet single unit criteria

single_use = [261, 262, 263, 264, 266, 268, 273, 276, 279]
df = df[df.propertylandusetypeid.isin(single_use)]
    
    
# Restrict df to only those properties with at least 1 bath & bed and >350 sqft area
df = df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0) & ((df.unitcnt<=1)|df.unitcnt.isnull()) & (df.calculatedfinishedsquarefeet>350)]
df.shape

(71355, 68)

2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

In [171]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .70):
    threshold = int(round(prop_required_column * len(df.index),0))
    df.dropna(axis = 1, thresh = threshold, inplace = True)
    threshold = int(round(prop_required_row * len(df.columns),0))
    df.dropna(axis = 0, thresh = threshold, inplace = True)
    return df

In [172]:
df = handle_missing_values(df)
df.shape

(71355, 34)

In [173]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71355 entries, 0 to 77573
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            71355 non-null  int64  
 1   parcelid                      71355 non-null  int64  
 2   bathroomcnt                   71355 non-null  float64
 3   bedroomcnt                    71355 non-null  float64
 4   buildingqualitytypeid         44841 non-null  float64
 5   calculatedbathnbr             71340 non-null  float64
 6   calculatedfinishedsquarefeet  71355 non-null  float64
 7   finishedsquarefeet12          71188 non-null  float64
 8   fips                          71355 non-null  float64
 9   fullbathcnt                   71340 non-null  float64
 10  heatingorsystemtypeid         46412 non-null  float64
 11  latitude                      71355 non-null  float64
 12  longitude                     71355 non-null  float64
 13  l

In [174]:
df.isnull().sum()

id                                  0
parcelid                            0
bathroomcnt                         0
bedroomcnt                          0
buildingqualitytypeid           26514
calculatedbathnbr                  15
calculatedfinishedsquarefeet        0
finishedsquarefeet12              167
fips                                0
fullbathcnt                        15
heatingorsystemtypeid           24943
latitude                            0
longitude                           0
lotsizesquarefeet                8034
propertycountylandusecode           0
propertylandusetypeid               0
propertyzoningdesc              26221
rawcensustractandblock              0
regionidcity                     1322
regionidcounty                      0
regionidzip                        40
roomcnt                             0
unitcnt                         26195
yearbuilt                          35
structuretaxvaluedollarcnt         75
taxvaluedollarcnt                   1
assessmentye

In [178]:
cols_to_remove = ['id', 'calculatedbathnbr', 'finishedsquarefeet12', 'fullbathcnt', 
                  'heatingorsystemtypeid', 'propertycountylandusecode', 
                  'propertylandusetypeid','propertyzoningdesc', 'heatingorsystemdesc',
                  'censustractandblock', 'propertylandusedesc', 'unitcnt', 
                  'buildingqualitytypeid']

In [179]:
def remove_columns(df, cols_to_remove):  
    df = df.drop(columns = cols_to_remove)
    return df

In [180]:
df = remove_columns(df, cols_to_remove)
df.isnull().sum()

parcelid                           0
bathroomcnt                        0
bedroomcnt                         0
calculatedfinishedsquarefeet       0
fips                               0
latitude                           0
longitude                          0
lotsizesquarefeet               8034
rawcensustractandblock             0
regionidcity                    1322
regionidcounty                     0
regionidzip                       40
roomcnt                            0
yearbuilt                         35
structuretaxvaluedollarcnt        75
taxvaluedollarcnt                  1
assessmentyear                     0
landtaxvaluedollarcnt              1
taxamount                          5
logerror                           0
transactiondate                    0
dtype: int64

In [181]:
# fill missing values with median null sizes
df.lotsizesquarefeet.fillna(7313, inplace = True)

df.shape

(71355, 21)

In [182]:
# Remove properties valued more than 5 million dollars
df = df[df.taxvaluedollarcnt < 5_000_000]

# Remove properties valued more than 8000 square feet
df = df[df.calculatedfinishedsquarefeet < 8000]

# Add column for counties
df['county'] = np.where(df.fips == 6037, 'Los_Angeles', np.where(df.fips == 6059, 'Orange', 
                                   'Ventura')) 

# I am going to drop the rest of nulls 
df = df.dropna()

df.shape

(69680, 22)

#### Mall Customers

In [183]:
def get_mall_customers():
    sql = 'select * from customers'
    url = get_db_url('mall_customers')
    mall_df = pd.read_sql(sql, url, index_col='customer_id')
    return mall_df

In [185]:
mall_df = get_mall_customers()

In [186]:
mall_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 1 to 200
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   gender          200 non-null    object
 1   age             200 non-null    int64 
 2   annual_income   200 non-null    int64 
 3   spending_score  200 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 7.8+ KB


In [187]:
mall_df.shape

(200, 4)

In [188]:
mall_df.describe()

,age,annual_income,spending_score
count,200.00,200.00,200.00
mean,38.85,60.56,50.20
std,13.97,26.26,25.82
min,18.00,15.00,1.00
25%,28.75,41.50,34.75
50%,36.00,61.50,50.00
75%,49.00,78.00,73.00
max,70.00,137.00,99.00


In [189]:
mall_df.isnull().sum()

gender            0
age               0
annual_income     0
spending_score    0
dtype: int64

In [190]:
def outlier_function(df, cols, k):
    for col in df[cols]:
        q1 = mall_df.annual_income.quantile(0.25)
        q3 = mall_df.annual_income.quantile(0.75)
        iqr = q3 - q1
        upper_bound =  q3 + k * iqr
        lower_bound =  q1 - k * iqr     
        df = df[(df[col] < upper_bound) & (df[col] > lower_bound)]
    return df

In [191]:
mall_df = outlier_function(mall_df, ['age', 'spending_score', 'annual_income'], 1.5)
mall_df.shape

(198, 4)

In [192]:
# encode the gender column

dummy_df = pd.get_dummies(mall_df.gender, drop_first=True)
dummy_df.head()

,Male
customer_id,
1,1
2,1
3,0
4,0
5,0


In [193]:
# concat the dummies df with original df. Drop the 'gender' column

mall_df = pd.concat([mall_df, dummy_df], axis=1).drop(columns = ['gender'])
mall_df.head()

,age,annual_income,spending_score,Male
customer_id,,,,
1,19,15,39,1
2,21,15,81,1
3,20,16,6,0
4,23,16,77,0
5,31,17,40,0


In [196]:
train, test = train_test_split(mall_df, train_size = 0.8, random_state = 123)
train, validate = train_test_split(train, train_size = 0.75, random_state = 123)

In [197]:
def min_max_scaler(train, valid, test):
    '''
    Uses the train & test datasets created by the split_my_data function
    Returns 3 items: mm_scaler, train_scaled_mm, test_scaled_mm
    This is a linear transformation. Values will lie between 0 and 1
    '''
    num_vars = list(train.select_dtypes('number').columns)
    scaler = MinMaxScaler(copy=True, feature_range=(0,1))
    train[num_vars] = scaler.fit_transform(train[num_vars])
    valid[num_vars] = scaler.transform(valid[num_vars])
    test[num_vars] = scaler.transform(test[num_vars])
    return scaler, train, valid, test

In [198]:
scaler, train_scaled, validate_scaled, test_scaled = min_max_scaler(train, validate, test)

In [199]:
train.shape

(118, 4)